<a href="https://colab.research.google.com/github/LucasMirandaVS/estudos_python/blob/main/Resolu%C3%A7%C3%A3o_Desafio_Eleflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Configurando o ambiente

In [1]:
import pandas as pd
import sqlite3

#Definindo os dados e o banco SQLite


Para facilitar a análise, todos os arquivos disponibilizados para esta parte do desafio foram colocados em uma única planilha Excel chamada *Dados Desafio Eleflow*.

In [2]:
# Função para carregar as abas do arquivo Excel para SQLite
def load_data_from_excel(file_path):
    # Carregar cada aba como um DataFrame
    board_games_df = pd.read_excel(file_path, sheet_name='Board Games Principal')
    designer_df = pd.read_excel(file_path, sheet_name='Designer_category')
    rating_df = pd.read_excel(file_path, sheet_name='Rating')

    # Conectar ao SQLite
    conn = sqlite3.connect('dados_desafio_eleflow.db')

    # Salvar os DataFrames no SQLite
    board_games_df.to_sql('Board_Games_Principal', conn, if_exists='replace', index=False)
    designer_df.to_sql('Designer_category', conn, if_exists='replace', index=False)
    rating_df.to_sql('Rating', conn, if_exists='replace', index=False)

    print("Dados carregados no banco de dados SQLite com sucesso.")

    return conn  # Retorna a conexão para uso nas consultas SQL



In [3]:
# Carregar os dados do Excel para o SQLite
conn = load_data_from_excel('/content/Dados Desafio Eleflow.xlsx')

Dados carregados no banco de dados SQLite com sucesso.


In [4]:
# Função para executar queries e mostrar resultados
def execute_query(query, conn):
    df = pd.read_sql_query(query, conn)
    return df

# Respondendo as perguntas

 1- Qual designer de jogos possui a maior média de avaliação dentre todos os seus jogos publicados?

In [5]:
query1 = '''
WITH designer_avg AS (
    SELECT d.designer, AVG(r.avg_rating) AS avg_designer_rating
    FROM Designer_category d
    JOIN Rating r ON d.game_id = r.game_id
    GROUP BY d.designer
)
SELECT designer, MAX(avg_designer_rating) AS highest_avg_rating
FROM designer_avg;
'''
resultado1 = execute_query(query1, conn)
print("Resultado da Query 1:")
print(resultado1)

Resultado da Query 1:
                  designer  highest_avg_rating
0  Sean Äaberg, Eric Radey            897594.0


2 - Quais os 5 jogos com durabilidade de até 120 min pior avaliados?

In [6]:
query2 = '''
SELECT b.names, r.avg_rating
FROM Board_Games_Principal b
JOIN Rating r ON b.game_id = r.game_id
WHERE b.max_time <= 120
ORDER BY r.avg_rating ASC
LIMIT 5;
'''
resultado2 = execute_query(query2, conn)
print("Resultado da Query 2:")
print(resultado2)

Resultado da Query 2:
                             names  avg_rating
0  Target Arnhem: Across 6 Bridges          65
1             Texas Glory: 1835-36          71
2                       Formule Dé          72
3                         Cauldron          72
4                              GEM         633


3 - Um cliente pretende comprar os 5 jogos melhor avaliados que atendam às seguintes características: seja da categoria ficção científica (“Science Fiction”), o máximo de jogadores seja 6 e possua a mecânica de controle de área (“Area control”) . Quais jogos o cliente deve por no carrinho?

In [21]:
query3 = '''
SELECT b.names, r.avg_rating
FROM Board_Games_Principal b
JOIN Rating r ON b.game_id = r.game_id
JOIN Designer_category d ON b.game_id = d.game_id
WHERE (d.category LIKE '%Science Fiction%' OR d.category LIKE '%Science Fiction,%')
AND b.max_players <= 6
AND (b.mechanic LIKE '%Area Control%' OR b.mechanic LIKE '%Area Control,%')
ORDER BY r.avg_rating DESC
LIMIT 5

'''

# Executando a query e mostrando os resultados
resultado3 = execute_query(query3, conn)
print("Resultado da Query 3:")
print(resultado3)


Resultado da Query 3:
                               names  avg_rating
0  Twilight Imperium: Fourth Edition      890337
1               Star Wars: Rebellion      847439
2                             Scythe      829158
3                           Renegade      827806
4          Malifaux (second edition)      826055


In [22]:
# Fechando a conexão com o banco de dados após o uso
conn.close()